# Resource Allocation in a Fast-Food Setting
---
### Problem Statement:
A fast-food restaurant has to decide how to allocate its resources among three main sections:

  1.  Cooking: Preparing the main items (burgers, wraps, etc.)
  2.  Assembly: Assembling the items together (adding sauces, making combos, etc.)
  3.  Serving: Taking orders and serving to the customers.

The goal is to maximize customer satisfaction, which depends on the efficiency of these sections. If any section is slow, it can bottleneck the whole process and decrease customer satisfaction.

**There are a total of 15 employees in the restaurant and each section needs at least 3 employees. Each employee can only be assigned to one section.**

Every employee has a different efficiency rate for each section, ranging from 0 to 1. A higher efficiency rate means the employee is more suitable for that section.

The customer satisfaction is calculated as the product of the average efficiency rates of the employees in each section.

---
### Objective: Allocate employees to sections in such a way that the customer satisfaction is maximized.
---
### Assumptions:
All employees work the same hours.
The efficiency of a section is simply the average of the efficiency of its employees.
The number of employees in a section doesn't affect its efficiency beyond the employee's individual efficiencies.

---
### Difference between Monte Carlo and Genetic Algorithm:

While both Monte Carlo and Genetic Algorithms involve randomness:

1. **Monte Carlo**:
    - Completely random sampling.
    - Each sample (or solution) is independent of the others.
    - May require many samples to get a good solution.
    - Does not "learn" from one iteration to the next.

2. **Genetic Algorithm**:
    - Begins with randomness but introduces "evolution."
    - Solutions improve over iterations.
    - Uses "mating" and "mutations" to produce new solutions.
    - Learns from previous solutions: good solutions produce offspring.

In the fast-food problem, Monte Carlo would generate a large number of assignment randomly. In contrast, the Genetic Algorithm would start with a set of random assufbnebt, then iteratively refine and evolve these assignment to find better ones over time.

In [1]:
# Sample efficiencies for 15 employees
efficiencies = [
    [0.7, 0.5, 0.8], [0.5, 0.9, 0.4], [0.8, 0.6, 0.7], [0.4, 0.7, 0.6], [0.9, 0.8, 0.5],
    [0.6, 0.5, 0.9], [0.5, 0.8, 0.6], [0.7, 0.9, 0.5], [0.6, 0.7, 0.8], [0.7, 0.8, 0.6],
    [0.8, 0.7, 0.7], [0.6, 0.6, 0.8], [0.7, 0.9, 0.5], [0.7, 0.8, 0.7], [0.6, 0.7, 0.9]
]

Let's introduce the following assumptions first.

In [7]:
pop_size=100
mutation_rate=0.20
generations=15
n=15

In [8]:
import numpy as np

# Setting a seed for reproducibility
np.random.seed(80)

def initialize_population(n, size):
    """
    Initialize a population of given size with random section assignments ensuring each section has at least 3 members.
    
    Parameters:
    - n (int): Number of employees.
    - size (int): Size of the population.
    
    Returns:
    - list: A list of lists representing the initial population with section assignments.
    """
    population = []
    for _ in range(size):
        while True:
            chromosome = list(np.random.choice([1, 2, 3], n))
            if all(chromosome.count(section) >= 3 for section in [1, 2, 3]):
                break
        population.append(chromosome)
    return population

def fitness(chromosome, efficiencies):
    """Calculate the fitness of a chromosome."""
    sections = [1, 2, 3]
    # For products, the initialized value is 1, instead of 0.
    satisfaction = 1
    # Looping per section
    for section in sections:
        # Finding the chromosomes belonging to that section
        employees_in_section = [i for i, s in enumerate(chromosome) if s == section]
        # Validity Checker
        #This code checks whether there is no employee assigned in that section - validity checker
        if not employees_in_section:
            return 0
        # Get the average efficiency per section
        avg_efficiency = np.mean([efficiencies[i][section-1] for i in employees_in_section])
        # Get the product
        satisfaction *= avg_efficiency
    return satisfaction

def crossover(parent1, parent2):
    """
    Perform a one-point crossover between two parent chromosomes.

    Parameters:
    - parent1 (list): The first parent chromosome.
    - parent2 (list): The second parent chromosome.

    Returns:
    - tuple: Two child chromosomes created from the parents.
    """
    idx = np.random.randint(1, len(parent1))
    child1 = list(np.concatenate((parent1[:idx], parent2[idx:])))
    child2 = list(np.concatenate((parent2[:idx], parent1[idx:])))
    return child1, child2


def mutate(chromosome):
    """
    Randomly change the section assignment of one employee in the chromosome.

    Parameters:
    - chromosome (list): A list indicating section assignments for each employee.

    Returns:
    - list: The mutated chromosome.
    """
    idx = np.random.randint(0, len(chromosome))
    chromosome[idx] = np.random.choice([1, 2, 3])
    return chromosome

            

def genetic_algorithm(efficiencies, n, population_size=pop_size, generations=15, mutation_prob=mutation_rate):
    """
    Optimize employee allocation using a genetic algorithm.
    
    Uses selection, crossover, and mutation to evolve a population towards better solutions.
    
    Parameters:
    - efficiencies (list): A list of efficiency values for each employee in each section.
    - n (int): Number of employees.
    - population_size (int, optional): Size of the population. Default is 100.
    - generations (int, optional): Number of generations. Default is 10.
    - mutation_prob (float, optional): Probability of mutation. Default is 0.2.
    
    Returns:
    - list: The best solution after all generations, representing section assignments for employees.
    """
    
    while True:
        # INITIALIZATION
        population = initialize_population(n, population_size)
        # LOOP TO EVOLVE THE POPULATION
        for gen in range(generations):
            # SORT FROM MOST FIT TO LEAST FIT
            population.sort(key=lambda x: -fitness(x, efficiencies))
            
            # Printing the maximum satisfaction for current generation
            best_fit = fitness(population[0], efficiencies)
            print(f"Generation {gen + 1}, Best Customer Satisfaction Score: {best_fit:.2f}")
            
            # GENERATION OF NEW POPULATION - PER GENERATION
            new_population = []
            
            while len(new_population) < population_size:
                
                # RANDOMLY CHOOSE FROM THE TOP 10 PARENTS
                parent_indices = np.random.choice(10, 2, replace=False)
                parent1, parent2 = population[parent_indices[0]], population[parent_indices[1]]
                
                # PERFORM CROSSOVER
                child1, child2 = crossover(parent1, parent2)
                
                # MUTATION AT RANDOM
                if np.random.rand() < mutation_prob:
                    child1 = mutate(child1)
                if np.random.rand() < mutation_prob:
                    child2 = mutate(child2)
                    
                new_population.extend([child1, child2])
            # ASSIGNING THE NEW POPULATION    
            population = new_population
        
        # Filtering out chromosomes that don't have at least 3 members in every section
        population = [chromosome for chromosome in population if all(chromosome.count(section) >= 3 for section in [1, 2, 3])]

        if not population:
            print("No viable population, rerunning evolution process")
            continue

        best_solution = max(population, key=lambda x: fitness(x, efficiencies))
        
        for section in [1, 2, 3]:
            employees_in_section = [i for i, s in enumerate(best_solution) if s == section]
            avg_efficiency = np.mean([efficiencies[i][section-1] for i in employees_in_section])
            print(f"Average efficiency for section {section}: {avg_efficiency:.2f}")

        print(f"Customer Satisfaction Score of the best solution: {best_fit:.2f}")

        return best_solution


# Run genetic algorithm
solution = genetic_algorithm(efficiencies, 15)
print("Optimal allocation (GA):", solution)


Generation 1, Best Customer Satisfaction Score: 0.45
Generation 2, Best Customer Satisfaction Score: 0.48
Generation 3, Best Customer Satisfaction Score: 0.52
Generation 4, Best Customer Satisfaction Score: 0.55
Generation 5, Best Customer Satisfaction Score: 0.55
Generation 6, Best Customer Satisfaction Score: 0.55
Generation 7, Best Customer Satisfaction Score: 0.55
Generation 8, Best Customer Satisfaction Score: 0.56
Generation 9, Best Customer Satisfaction Score: 0.56
Generation 10, Best Customer Satisfaction Score: 0.58
Generation 11, Best Customer Satisfaction Score: 0.58
Generation 12, Best Customer Satisfaction Score: 0.58
Generation 13, Best Customer Satisfaction Score: 0.58
Generation 14, Best Customer Satisfaction Score: 0.58
Generation 15, Best Customer Satisfaction Score: 0.60
Average efficiency for section 1: 0.83
Average efficiency for section 2: 0.83
Average efficiency for section 3: 0.84
Customer Satisfaction Score of the best solution: 0.60
Optimal allocation (GA): [3